In [ ]:
import os
import polars as pl

In [ ]:
# Set Polars to show full DataFrame output (no truncation)
pl.Config.set_tbl_formatting("ASCII_FULL")   # or "UTF8_FULL" if you prefer UTF8 borders
pl.Config.set_tbl_rows(1000)      # Increase max rows (default is 10)
pl.Config.set_tbl_cols(50)        # Increase max columns (default is 10)

# Now your print(group_df) will show full content


# read data from bronze table into a DataFrame
file_path = "data/bronze_df.parquet"

# Load the Parquet file into a Polars DataFrame
df = pl.read_parquet(file_path)

columns = df.columns
print(columns)

In [ ]:
# overview of the data
all_code_groups = {
    "Gender Codes": {
        226228: "Gender"
    },
    "Age Codes": {
        226984: "Apache IV Age"
    },
    "Confusion / Mental Instability": {
        228395: "Orientation to Place",
        228394: "Orientation to Person",
        229381: "Orientation",
        223898: "Orientation",
        228396: "Orientation to Time",
        226104: "Level of Consciousness",
        229382: "Orientation Score",
        228688: "Delirium",
        2930: "Delirium due to conditions classified elsewhere",
        2903: "Senile dementia with delirium",
        2931: "Subacute delirium",
        2982: "Reactive confusion",
        29281: "Drug-induced delirium",
        29011: "Presenile dementia with delirium",
        78097: "Altered mental status",
        29041: "Vascular dementia, with delirium",
        "F05": "Delirium due to known physiological condition"
    },
    "Respiratory Rate": {
        230040: "Paradoxical breathing"
    },
    "Blood Pressure": {
        225309: "ART BP Systolic",
        227243: "Manual Blood Pressure Systolic Right",
        220179: "Non Invasive Blood Pressure Systolic",
        220050: "Arterial Blood Pressure Systolic",
        224167: "Manual Blood Pressure Systolic Left"
    },
    "Temperature": {
        224027: "Skin Temperature",
        223761: "Temperature Fahrenheit",
        223762: "Temperature Celsius",
        226329: "Blood Temperature CCO (C)",
        50825: "Temperature Blood Blood Gas"
    },
    "Pulse Rates": {
        229770: "Resting Pulse Rate (COWS)",
        223942: "Graft/Flap Pulse",
        223936: "Radial Pulse R",
        223948: "Radial Pulse L",
        223941: "Popliteal Pulse R",
        223946: "Popliteal Pulse L",
        223949: "Ulnar Pulse L",
        223945: "Femoral Pulse L",
        223939: "Brachial Pulse R",
        223944: "Brachial Pulse L",
        223940: "Femoral Pulse R",
        223938: "Ulnar Pulse R"
    },
    "BUN": {
        225624: "BUN",
        51842: "Bun"
    },
    "pH": {
        50820: "pH",
        223830: "PH (Arterial)"
    },
    "Sodium": {
        220645: "Sodium (serum)",
        226534: "Sodium (whole blood)",
        228389: "Sodium (serum) (soft)",
        228390: "Sodium (whole blood) (soft)",
        50983: "Sodium",
        52623: "Sodium"
    },
    "Glucose": {
        50809: "Glucose",
        50931: "Glucose",
        52569: "Glucose",
        226537: "Glucose (whole blood)",
        225664: "Glucose finger stick (range 70-100)",
        220621: "Glucose (serum)",
        228388: "Glucose (whole blood) (soft)"
    },
    "Hematocrit": {
        52028: "Hematocrit Blood",
        51638: "Hematocrit Blood",
        51639: "Hematocrit Blood",
        51221: "Hematocrit Blood",
        226540: "Hematocrit (whole blood - calc)",
        220545: "Hematocrit (serum)"
    },
    "Oxygen Saturation": {
        220227: "Arterial O2 Saturation",
        220277: "O2 saturation pulseoxymetry",
        223835: "Inspired O2 Fraction",
        50817: "Oxygen Saturation"
    },
    "Pleural Effusion": {
        51181: "Malignant pleural effusion",
        5119: "Unspecified pleural effusion",
        "J910": "Malignant pleural effusion",
        "J918": "Pleural effusion in other conditions classified elsewhere",
        "J91": "Pleural effusion in conditions classified elsewhere"
    }
}

final_data = {}
missing_in_group = {}

for group_name, codes in all_code_groups.items():
    group_data = []
    print(f"\nProcessing {group_name}:")

    for code, description in codes.items():
        # Determine type of code and filter accordingly
        if isinstance(code, int):
            filtered_df = df.filter(pl.col("itemid") == code)
        else:
            # code is string, so cast itemid to string for comparison
            filtered_df = df.filter(pl.col("itemid").cast(pl.Utf8) == code)

        # Check if filtered_df is empty
        if filtered_df.is_empty():
            print(f"No data found for {description} (code {code}). Skipping.")
            missing_in_group[code] = description
            continue
        
        # Inspect unique units
        unique_units = filtered_df.select(pl.col("valueuom").unique())
        unique_units_list = unique_units["valueuom"].to_list()

        # Calculate range of numerical values if column exists
        if "valuenum" in filtered_df.columns and not filtered_df["valuenum"].is_empty():
            value_stats = filtered_df.select([
                pl.col("valuenum").min().alias("min_value"),
                pl.col("valuenum").max().alias("max_value")
            ])
            min_value = value_stats["min_value"][0]
            max_value = value_stats["max_value"][0]
            print(f"Range of numerical values for {description} (code {code}): {min_value} to {max_value}")
        else:
            print(f"No numerical values found for {description} (code {code}).")

        # Print unique units
        print(f"Unique units for {description} (code {code}): {unique_units_list}")

        # Append filtered data
        group_data.append(filtered_df)

    # Combine the group's filtered data into a single DataFrame if any
    final_data[group_name] = pl.concat(group_data) if group_data else None

print(missing_in_group)

# Print final DataFrames for each group
#for group_name, group_df in final_data.items():
#    print(f"\nFinal DataFrame for {group_name}:")
#    if group_df is not None:
#        print(group_df)
#    else:
#        print("No data found.")

In [ ]:
# 50817 Oxygen Saturation → korrekt für Partial Pressure of Arterial Oxygen
# 220277 O2 saturation pulseoxymetry → ist Sauerstoffsättigung, nicht arterieller Sauerstoffpartialdruck
# Die verbleibenden Codes werden manuell überprüft, ob sie für die Kalkulation der Scores gebraucht werden, 
# zudem wird recherchiert ob die Range der Werte Sinn ergibt, oder ob in einem nächsten Schritt noch nach 
# Ausreißern gesucht werden muss

In [ ]:
# Remove keys in data_two from all_code_groups
filtered_code_groups = {
    group: {
        key: value
        for key, value in codes.items()
        if key not in missing_in_group
    }
    for group, codes in all_code_groups.items()
}

# Remove empty groups
filtered_code_groups = {group: codes for group, codes in filtered_code_groups.items() if codes}

filtered_code_groups

In [ ]:
# Manually editing the result to add the concept names and ids - checking value range in overview output
clean_codes = {

    # this part is going to be excluded, since the focus of the orientation tests is more towards determining dementia etc
    #"Confusion / Mental Instability": {
    #    228395: ("Orientation to Place", 45881443, "Confusion"),
    #    228394: ("Orientation to Person", 45881443, "Confusion"),
    #    229381: ("Orientation", 45881443, "Confusion"),
    #    228396: ("Orientation to Time", 45881443, "Confusion")
    #},

    # all correct
    "Blood Pressure": {
        225309: ("ART BP Systolic", 4152194, "Systolic blood pressure"),
        220179: ("Non Invasive Blood Pressure Systolic", 4152194, "Systolic blood pressure"),
        220050: ("Arterial Blood Pressure Systolic", 4152194, "Systolic blood pressure")
    },

    # skin temperature excluded, since no numeric results, Fahrenheit temp can be converted 
    # (is probably redundant and will therefore be filtered out)
    "Temperature": {
        # 224027: ("Skin Temperature", 36716470, "Temperature"),
        # 223761: ("Temperature Fahrenheit", 36716470, "Temperature"),
        223762: ("Temperature Celsius", 36716470, "Temperature")
    },

    # INVESTIGATE in overview only none to none value ranges, will be investigated in next step
    # after investigation - no numeric values, only if pulse is present, will be excluded
    #"Pulse Rates": {
    #    223936: ("Radial Pulse R", 4224504, "Pulse"),
    #    223948: ("Radial Pulse L", 4224504, "Pulse"),
    #    223938: ("Ulnar Pulse R", 4224504, "Pulse")
    #},

    # all correct
    "BUN": {
        225624: ("BUN", 4017361, "Blood urea nitrogen measurement")
    },

    # all correct
    "pH": {
        223830: ("PH (Arterial)", 4097822, "pH measurement, arterial")
    },

    # looking good, mEq/L instead of mmol/L, after some research, seems to be the same for Sodium
    "Sodium": {
        220645: ("Sodium (serum)", 4097430, "Sodium measurement"),
        226534: ("Sodium (whole blood)", 4097430, "Sodium measurement")
    },

    # correct, for finger there is no unit, so might exclude that
    "Glucose": {
        226537: ("Glucose (whole blood)", 4149519, "Glucose measurement"),
        #225664: ("Glucose finger stick (range 70-100)", 4149519, "Glucose measurement"),
        220621: ("Glucose (serum)", 4149519, "Glucose measurement")
    },

    # all correct
    "Hematocrit": {
        226540: ("Hematocrit (whole blood - calc)", 4151358, "Hematocrit determination"),
        220545: ("Hematocrit (serum)", 4151358, "Hematocrit determination")
    }

    # all measurements that remain are not referring to Partial pressure of arterial oxygen, but other measurements
    # of oxygen and are therfore excluded
    #"Oxygen Saturation": {
    #    220227: ("Arterial O2 Saturation", 4103460, "Partial pressure of arterial oxygen"),
    #    220277: ("O2 saturation pulseoxymetry", 4103460, "Partial pressure of arterial oxygen"),
    #    223835: ("Inspired O2 Fraction", 4103460, "Partial pressure of arterial oxygen")
    #}
}

# these are also checked later
important_labels = {
    ("gender", 4135376, "Gender"),
    ("anchor_age", 4265453, "Age"),
    ("admission_location", 45883663, "Nursing Home")
}



In [ ]:
# get rows with labels: "gender", "anchor_age", "admission_location"

filtered_df = df.filter(pl.col("label") == "gender")

# Calculating the lexicographical range of values in the "valuenum" column
valuenum_min = filtered_df["valuenum"].min()
valuenum_max = filtered_df["valuenum"].max()

# Print results
print("Filtered DataFrame:")
print(filtered_df)

print(f"\nLexicographical Range of 'valuenum' column: (Min: {valuenum_min}, Max: {valuenum_max})")



In [ ]:

filtered_df = df.filter(pl.col("label") == "anchor_age")

# Calculating the lexicographical range of values in the "valuenum" column
valuenum_min = filtered_df["valuenum"].min()
valuenum_max = filtered_df["valuenum"].max()

# Print results
print("Filtered DataFrame:")
print(filtered_df)

print(f"\nLexicographical Range of 'valuenum' column: (Min: {valuenum_min}, Max: {valuenum_max})")

In [ ]:

filtered_df = df.filter(pl.col("label") == "admission_location")

# Calculating the lexicographical range of values in the "valuenum" column
valuenum_min = filtered_df["valuenum"].min()
valuenum_max = filtered_df["valuenum"].max()

# Print results
print("Filtered DataFrame:")
print(filtered_df)

print(f"\nLexicographical Range of 'valuenum' column: (Min: {valuenum_min}, Max: {valuenum_max})")

All results here look good

TO check:

INVESTIGATE in overview only none to none value ranges, will be investigated in next step

    "Pulse Rates": {
        223936: ("Radial Pulse R", 4224504, "Pulse"),
        223948: ("Radial Pulse L", 4224504, "Pulse"),
        223938: ("Ulnar Pulse R", 4224504, "Pulse")
    },

225664: ("Glucose finger stick (range 70-100)", 4149519, "Glucose measurement"),

In [ ]:
filtered_df = df.filter(pl.col("itemid") == 225664)

# Calculating the lexicographical range of values in the "valuenum" column
valuenum_min = filtered_df["valuenum"].min()
valuenum_max = filtered_df["valuenum"].max()

# Print results
print("Filtered DataFrame:")
print(filtered_df)

print(f"\nLexicographical Range of 'valuenum' column: (Min: {valuenum_min}, Max: {valuenum_max})")

# note - values get too high, makes sense since its measured on the finger - will still exclude it, 
# since there is no unit given, so the values cannot be interpreted

In [ ]:
filtered_df = df.filter(pl.col("itemid") == 223948) #223948, 223938 223936))

# Calculating the lexicographical range of values in the "valuenum" column
valuenum_min = filtered_df["valuenum"].min()
valuenum_max = filtered_df["valuenum"].max()

# Print results
print("Filtered DataFrame:")
print(filtered_df)

print(f"\nLexicographical Range of 'valuenum' column: (Min: {valuenum_min}, Max: {valuenum_max})")

In [ ]:
# Now since everything is filtered, these are our lists
print(clean_codes)
print(important_labels)

In [ ]:
# pivoted table (did it manually)
data_labels = [
    ('gender', 'anchor_age', 'admission_location'),
    (4135376, 4265453, 45883663),
    ('Gender', 'Age', 'Nursing Home')
]

# Create the Polars DataFrame
df_labels = pl.DataFrame(data_labels, schema=[("label", pl.Utf8), ("concept_id", pl.Int64), ("concept_name", pl.Utf8)], strict=False)

df_labels

In [ ]:
data_one = [(k, int(v[1]), v[2]) for group in clean_codes.values() for k, v in group.items()]
df_one = pl.DataFrame(data_one, schema=[("itemid", pl.Int64), ("new_concept_id", pl.Int64), ("new_concept_name", pl.Utf8)])

df_one

In [ ]:
# now using left joins to add the new colums to our dataframe
joined_df = df.join(df_labels, on="label", how="left")


columns = joined_df.columns
print(columns)

# join the second table
joined_df = joined_df.join(df_one, on="itemid", how="left")

columns = joined_df.columns
print(columns)

# merge the columns 
joined_df = joined_df.with_columns([
    pl.when(pl.col("concept_id").is_null())
    .then(pl.col("new_concept_id"))
    .otherwise(pl.col("concept_id"))
    .alias("concept_id"),
    pl.when(pl.col("concept_name").is_null())
    .then(pl.col("new_concept_name"))
    .otherwise(pl.col("concept_name"))
    .alias("concept_name")
])

# Drop the temporary columns if they are no longer needed
joined_df = joined_df.drop(["new_concept_id", "new_concept_name"])

# Filter out rows where concept_id is null - these are the values that we had to filter out since they arent needed for our calculation
result = joined_df.filter(~pl.col("concept_id").is_null())
result



In [ ]:
# Save as parquet file
result.write_parquet("data/silver_df.parquet")